In [48]:
import pandas as pd

# URL of the raw csv file in the GitHub repository
url = 'https://raw.githubusercontent.com/hcagatayyilmaz/data-literacy-extreme-wealth-analysis/main/billionaires.csv'

# Use pandas to import the csv file
df = pd.read_csv(url)

In [49]:
df.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


Data preprocessing- copied from Kuebra's colab

In [50]:
df.columns

Index(['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship', 'organization',
       'selfMade', 'status', 'gender', 'birthDate', 'lastName', 'firstName',
       'title', 'date', 'state', 'residenceStateRegion', 'birthYear',
       'birthMonth', 'birthDay', 'cpi_country', 'cpi_change_country',
       'gdp_country', 'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country'],
      dtype='object')

In [51]:
# Check for null values in each column
null_values = df.isnull().any()

# Count null values in each column
null_counts = df.isnull().sum()

null_percent = (round(null_counts/len(df),2)*100).astype(int)

# Create a new DataFrame
null_info_df = pd.DataFrame({
    'Has_Null': null_values,
    'Null_Count': null_counts,
    'Null_Percent': null_percent
})

# Display the result
print(null_info_df.sort_values(by='Null_Count', ascending=False))

                                            Has_Null  Null_Count  Null_Percent
organization                                    True        2315            88
title                                           True        2301            87
residenceStateRegion                            True        1893            72
state                                           True        1887            71
cpi_change_country                              True         184             7
cpi_country                                     True         184             7
tax_revenue_country_country                     True         183             7
total_tax_rate_country                          True         182             7
life_expectancy_country                         True         182             7
gross_tertiary_education_enrollment             True         182             7
gross_primary_education_enrollment_country      True         181             7
latitude_country                                True

In [52]:
# Drop the columns with almost only have null values
df.drop(["organization", "title", "state", "residenceStateRegion"], axis=1, inplace=True)

In [53]:
# Check for duplicates
df.duplicated().any()

False

In [54]:
# Convert 'gdp_country' to float by removing '$' and ','
df['gdp_country'] = df['gdp_country'].str.replace('$','').replace(',','', regex=True).astype(float)
print(df['gdp_country'].dtype)

float64


C:\Users\pulki\AppData\Local\Temp\ipykernel_1476\507803359.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['gdp_country'] = df['gdp_country'].str.replace('$','').replace(',','', regex=True).astype(float)


In [55]:
# To see whether we can fill null values in country column usign latitude and logitude columns:
condition_country = (df['country'].isnull()) & (df['longitude_country'].notnull()) & (df['latitude_country'].notnull())

# Check if there are any rows that satisfy the conditions
condition_country.any()

False

In [56]:
# Since country is an important feature for our analysis. We drop the rows with null country values
df.dropna(subset=['country'], inplace=True)

In [57]:
# To see if we can fill out null values in age column using birthDate or birthYear columns
condition_age = ((df['age'].isnull()) & ((df['birthDate'].notnull()) | df['birthYear'].notnull()))

# Check if there are any rows that satisfy the conditions
condition_age.any()

False

In [58]:
# Drop unnecessary columns
df.drop(["birthDay", "birthMonth", "birthYear"], axis=1, inplace=True)

In [59]:
# Convert 'birthDate' to datetime
df['birthDate'] = pd.to_datetime(df['birthDate'], errors='coerce')

In [60]:
# Specify the columns with missing values and the country column
columns_to_fill = ['cpi_country','cpi_change_country',
                   'gdp_country', 'gross_tertiary_education_enrollment',
                   'gross_primary_education_enrollment_country',
                   'life_expectancy_country', 'tax_revenue_country_country',
                   'total_tax_rate_country', 'population_country',
                   'latitude_country', 'longitude_country']

# Identify unique countries with complete information
complete_countries = df.dropna(subset=columns_to_fill + ['country'])[['country'] + columns_to_fill].drop_duplicates()

# Group by 'country' and fill missing values with corresponding country's values
df[columns_to_fill] = df.groupby('country')[columns_to_fill].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

# Drop duplicate rows based on all columns
df = df.drop_duplicates()

# Reset index after dropping duplicates
df = df.reset_index(drop=True)

# Missing values after filling
df[columns_to_fill].isnull().sum()

cpi_country                                   146
cpi_change_country                            146
gdp_country                                   126
gross_tertiary_education_enrollment           144
gross_primary_education_enrollment_country    143
life_expectancy_country                       144
tax_revenue_country_country                   145
total_tax_rate_country                        144
population_country                            126
latitude_country                              126
longitude_country                             126
dtype: int64

In [61]:
# Drop the remaining null values
df.dropna(inplace=True)

In [62]:
# Count the total number of rows with at least one missing value
total_missing_rows = (df.isnull().any(axis=1)).sum()

# Print the result
print(f'Total rows with missing values: {total_missing_rows}')

Total rows with missing values: 0


In [63]:
df.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,2.715518e+12,65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,2.142770e+13,88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,2.142770e+13,88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,2.142770e+13,88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,2.142770e+13,88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


In [64]:
df.shape

(2369, 28)

In [65]:
df.columns

Index(['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship', 'selfMade',
       'status', 'gender', 'birthDate', 'lastName', 'firstName', 'date',
       'cpi_country', 'cpi_change_country', 'gdp_country',
       'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country'],
      dtype='object')

Let’s calculate the GDP per capita and drop the ‘gdp_country’ and ‘population_country’ columns:

In [66]:
# Calculate GDP per capita
df['gdp_per_capita'] = df['gdp_country'] / df['population_country']

# Drop 'gdp_country' and 'population_country' columns
df = df.drop(['gdp_country', 'population_country'], axis=1)

In [67]:
print(df['selfMade'].unique())
print(df['gender'].unique())


[False  True]
['M' 'F']


Next, let’s perform one-hot encoding on the categorical variables:

In [69]:
df['selfMade'] = df['selfMade'].replace({True: 'Yes', False: 'No'})
df['gender'] = df['gender'].replace({'M': 'Male', 'F': 'Female'})

df = pd.get_dummies(df, columns=['selfMade', 'gender', 'industries'])


Null hypothesis: ‘industries’ do not have a significant effect on ‘finalWorth’.


Now, let’s define the predictors for each model and the target variable:

In [75]:
# Define the predictors for the first model
predictors_model1 = ['age', 'selfMade_Yes', 'selfMade_No', 'gender_Male', 'gender_Female', 
                     'gdp_per_capita', 'tax_revenue_country_country', 'total_tax_rate_country']

# Define the predictors for the second model
predictors_model2 = predictors_model1 + [col for col in df.columns if 'industries_' in col]

# Define the target variable
target = 'finalWorth'

Finally, let’s fit the models and perform an ANOVA test to compare them:

In [76]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Fit the first model
model1 = sm.OLS(df[target], sm.add_constant(df[predictors_model1])).fit()

# Fit the second model
model2 = sm.OLS(df[target], sm.add_constant(df[predictors_model2])).fit()

# Perform an ANOVA test
anova_results = sm.stats.anova_lm(model1, model2)
print(anova_results)

   df_resid           ssr  df_diff       ss_diff         F    Pr(>F)
0    2362.0  2.477795e+11      0.0           NaN       NaN       NaN
1    2345.0  2.431627e+11     17.0  4.616821e+09  2.619027  0.000315


Based on these results, it appears that Model 1 (which includes ‘industries’ as a predictor) provides a significantly better fit to the data than Model 0, as indicated by the smaller ssr, the larger F-statistic, and the small p-value.

Null hypothesis: ‘industries’ do not have a significant effect on ‘finalWorth’.


Let's check if adding age as a predictor improves model fit

In [77]:
# Define the predictors for the first model (without 'age' and 'country')
predictors_model1 = ['selfMade_Yes', 'selfMade_No', 'gender_Male', 'gender_Female', 
                     'gdp_per_capita', 'tax_revenue_country_country', 'total_tax_rate_country']

# Define the predictors for the second model (with 'age' but without 'country')
predictors_model2 = predictors_model1 + ['age']


In [78]:
# Fit the first model
model1 = sm.OLS(df[target], sm.add_constant(df[predictors_model1])).fit()

# Fit the second model
model2 = sm.OLS(df[target], sm.add_constant(df[predictors_model2])).fit()

# Perform an ANOVA test
anova_results = sm.stats.anova_lm(model1, model2)
print(anova_results)


   df_resid           ssr  df_diff       ss_diff         F    Pr(>F)
0    2363.0  2.484486e+11      0.0           NaN       NaN       NaN
1    2362.0  2.477795e+11      1.0  6.691178e+08  6.378479  0.011616


Based on these results, it appears that adding ‘age’ as a predictor (Model 1) provides a significantly better fit to the data than the model without ‘age’ (Model 0), as indicated by the smaller ssr, the larger F-statistic, and the small p-value.